In [26]:
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz

In [27]:
USER_REVIEWS_PATH = '../data/video-games/all_games.csv'
SALES_PATH = '../data/video-games/vgsales.csv'

user_reviews_df = pd.read_csv(USER_REVIEWS_PATH)
sales_df = pd.read_csv(SALES_PATH)

In [28]:
user_reviews_df.head()

,name,platform,release_date,summary,meta_score,user_review
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998","As a young boy, Link is tricked by Ganondorf, ...",99,9.1
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",As most major publishers' development efforts ...,98,7.4
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",[Metacritic's 2008 PS3 Game of the Year; Also ...,98,7.7
3,SoulCalibur,Dreamcast,"September 8, 1999","This is a tale of souls and swords, transcendi...",98,8.4
4,Grand Theft Auto IV,Xbox 360,"April 29, 2008",[Metacritic's 2008 Xbox 360 Game of the Year; ...,98,7.9


In [29]:
sales_df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


### Things to do:
1. Remove unneccesary columns
2. Match shared columns names
3. Remove duplicated values
4. Match the values in the "Plarform" column for both data sets
5. Remove unmatched rows on both data sets.
6. Get A combined data set
7. Add User Review Col To Sales Data Frame
8. Make DS preproccesing for all 3 data sets

#### 1. Remove Unneccesary Columns:

In [30]:
def remove_unneccesary_columns(user_reviews_df, sales_df):
    t_sales_df = sales_df.drop(columns=["Rank"])
    t_user_reviews_df = user_reviews_df.drop(columns=["summary"])
    return t_user_reviews_df, t_sales_df

In [31]:
user_reviews_df, sales_df = remove_unneccesary_columns(user_reviews_df=user_reviews_df, sales_df=sales_df)

In [32]:
user_reviews_df.head()

,name,platform,release_date,meta_score,user_review
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998",99,9.1
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",98,7.4
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",98,7.7
3,SoulCalibur,Dreamcast,"September 8, 1999",98,8.4
4,Grand Theft Auto IV,Xbox 360,"April 29, 2008",98,7.9


In [33]:
sales_df.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


#### 2. Match shared columns names:

In [34]:
def match_shared_columns_names(user_reviews_df):
    t_user_reviews_df = user_reviews_df.rename(columns={"name": "Name"})
    t_user_reviews_df = t_user_reviews_df.rename(columns={"platform": "Platform"})
    return t_user_reviews_df


In [35]:
user_reviews_df = match_shared_columns_names(user_reviews_df=user_reviews_df)

In [36]:
user_reviews_df.head()

,Name,Platform,release_date,meta_score,user_review
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998",99,9.1
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",98,7.4
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",98,7.7
3,SoulCalibur,Dreamcast,"September 8, 1999",98,8.4
4,Grand Theft Auto IV,Xbox 360,"April 29, 2008",98,7.9


#### 3. Match the values in the "Plarform" column for both data sets

First we need to understand the mapping between each platform name:

In [37]:
user_reviews_platforms = user_reviews_df['Platform'].unique()
print(user_reviews_platforms)

[' Nintendo 64' ' PlayStation' ' PlayStation 3' ' Dreamcast' ' Xbox 360'
 ' Wii' ' Xbox One' ' PC' ' Switch' ' PlayStation 2' ' PlayStation 4'
 ' GameCube' ' Xbox' ' Wii U' ' Game Boy Advance' ' 3DS' ' Xbox Series X'
 ' DS' ' PlayStation Vita' ' PlayStation 5' ' PSP' ' Stadia']


In [38]:
sales_platforms = sales_df['Platform'].unique()
print(sales_platforms)

['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' '3DS' 'PS4' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'GC' 'WiiU' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX']


Using LLM we understang that the proper match is:
| **Dataset 1 (Standardized Names)** | **Dataset 2 (Abbreviations)** |
|-------------------------------------|------------------------------|
| Nintendo 64                        | N64                          |
| PlayStation                         | PS                           |
| PlayStation 2                       | PS2                          |
| PlayStation 3                       | PS3                          |
| PlayStation 4                       | PS4                          |
| PlayStation 5                       | (No match)                   |
| PlayStation Vita                    | PSV                          |
| PSP                                  | PSP                          |
| Dreamcast                           | DC                           |
| Xbox                                 | XB                           |
| Xbox 360                             | X360                         |
| Xbox One                             | XOne                         |
| Xbox Series X                        | (No match)                   |
| Wii                                  | Wii                          |
| Wii U                                | WiiU                         |
| Switch                               | (No match)                   |
| PC                                   | PC                           |
| GameCube                             | GC                           |
| Game Boy Advance                     | GBA                          |
| 3DS                                  | 3DS                          |
| DS                                   | DS                           |
| Stadia                               | (No match)                   |

In [39]:
def map_platform_names(user_reviews_df, sales_df):

    # Define platform mapping based on the table
    platform_mapping = {
        "N64": "Nintendo 64",
        "PS": "PlayStation",
        "PS2": "PlayStation 2",
        "PS3": "PlayStation 3",
        "PS4": "PlayStation 4",
        "PSV": "PlayStation Vita",
        "PSP": "PSP",
        "DC": "Dreamcast",
        "XB": "Xbox",
        "X360": "Xbox 360",
        "XOne": "Xbox One",
        "Wii": "Wii",
        "WiiU": "Wii U",
        "PC": "PC",
        "GC": "GameCube",
        "GBA": "Game Boy Advance",
        "3DS": "3DS",
        "DS": "DS"
    }
    
    # Create a new DataFrame (immutable approach)
    sales_df_updated = sales_df.copy()

    # Apply mapping while keeping unmatched values unchanged
    sales_df_updated["Platform"] = sales_df_updated["Platform"].map(platform_mapping).fillna(sales_df_updated["Platform"])
    
    #strip to avoid problems later
    sales_df_updated["Platform"] = sales_df_updated["Platform"].str.strip()
    user_reviews_df['Platform'] = user_reviews_df['Platform'].str.strip()

    return user_reviews_df, sales_df_updated

In [40]:
user_reviews_df, sales_df = map_platform_names(user_reviews_df=user_reviews_df ,sales_df=sales_df)

#### 4. Remove unmatched rows on both data sets.

In [41]:
def remove_unmatched_rows(user_reviews_df, sales_df):
    # Get matching 'Name' and 'Platform' pairs
    common_keys = set(user_reviews_df[['Name', 'Platform']].apply(tuple, axis=1)) & \
                  set(sales_df[['Name', 'Platform']].apply(tuple, axis=1))

    # Create new DataFrames with only matching rows
    filtered_user_reviews_df = user_reviews_df[user_reviews_df[['Name', 'Platform']].apply(tuple, axis=1).isin(common_keys)].copy()
    filtered_sales_df = sales_df[sales_df[['Name', 'Platform']].apply(tuple, axis=1).isin(common_keys)].copy()

    return filtered_user_reviews_df, filtered_sales_df

In [47]:
user_reviews_df, sales_df = remove_unmatched_rows(user_reviews_df=user_reviews_df, sales_df=sales_df)

In [48]:
user_reviews_df.head()

,Name,Platform,release_date,meta_score,user_review
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998",99,9.1
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",98,7.4
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",98,7.7
3,SoulCalibur,Dreamcast,"September 8, 1999",98,8.4
4,Grand Theft Auto IV,Xbox 360,"April 29, 2008",98,7.9


In [49]:
sales_df.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.38,9.23,6.50,2.90,30.01
7,Wii Play,Wii,2006.0,Misc,Nintendo,14.03,9.20,2.93,2.85,29.02


In [45]:
sales_df[sales_df['Name'] == 'The Legend of Zelda: Ocarina of Time']

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
94,The Legend of Zelda: Ocarina of Time,Nintendo 64,1998.0,Action,Nintendo,4.10,1.89,1.45,0.16,7.60
268,The Legend of Zelda: Ocarina of Time,3DS,2011.0,Action,Nintendo,2.03,1.27,0.62,0.30,4.21


In [46]:
user_reviews_df[user_reviews_df['Name'] == 'The Legend of Zelda: Ocarina of Time']

,Name,Platform,release_date,meta_score,user_review
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998",99,9.1


In [51]:
print(len(sales_df), len(user_reviews_df))

6135 6169


In [58]:
duplicate_pairs = user_reviews_df.groupby(["Platform", "Name"]).size().reset_index(name="count")
duplicate_pairs = duplicate_pairs[duplicate_pairs["count"] > 1].drop(columns=["count"])

In [59]:
print(duplicate_pairs)

              Platform                                       Name
534                 DS            The Lord of the Rings: Conquest
878   Game Boy Advance                            WTA Tour Tennis
1177          GameCube                                Teen Titans
1307                PC           Agatha Christie: The ABC Murders
1435                PC                                     DiRT 2
1536                PC     Harry Potter and the Half-Blood Prince
1537                PC  Harry Potter and the Order of the Phoenix
1630                PC                Need for Speed: Hot Pursuit
1701                PC                                   Sacred 3
1734                PC                               Spider-Man 3
1751                PC                                  Stormrise
1761                PC                                    Syberia
1775                PC                         The Cursed Crusade
2384     PlayStation 2           18 Wheeler: American Pro Trucker
2475     P

In [55]:
duplicate_pairs = sales_df.groupby(["Platform", "Name"]).size().reset_index(name="count")
duplicate_pairs = duplicate_pairs[duplicate_pairs["count"] > 1].drop(columns=["count"])

In [56]:
print(len(duplicate_pairs))

3


In [57]:
print(duplicate_pairs)

           Platform                         Name
1631             PC  Need for Speed: Most Wanted
3720  PlayStation 3                Madden NFL 13
5773       Xbox 360  Need for Speed: Most Wanted


#### 6. Get A Combined Data Set

In [61]:
def combine_data_set(user_reviews_df, sales_df, cols):
    merged_df = pd.merge(user_reviews_df, sales_df, on=cols, how="inner")
    return merged_df

In [62]:
combined_df = combine_data_set(user_reviews_df=user_reviews_df, sales_df=sales_df, cols=['Name', 'Platform'])

In [63]:
combined_df.head()

,Name,Platform,release_date,meta_score,user_review,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998",99,9.1,1998.0,Action,Nintendo,4.10,1.89,1.45,0.16,7.60
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",98,7.4,2000.0,Sports,Activision,3.05,1.41,0.02,0.20,4.68
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",98,7.7,2008.0,Action,Take-Two Interactive,4.76,3.76,0.44,1.62,10.57
3,SoulCalibur,Dreamcast,"September 8, 1999",98,8.4,1999.0,Fighting,Namco Bandai Games,0.00,0.00,0.34,0.00,0.34
4,Grand Theft Auto IV,Xbox 360,"April 29, 2008",98,7.9,2008.0,Action,Take-Two Interactive,6.76,3.10,0.14,1.03,11.02


In [64]:
def append_user_review_col_to_sales_df(user_reviews_df, sales_df):
    updated_user_review_df = sales_df.merge(user_reviews_df[['Name', 'Platform','user_review']], on=['Name', 'Platform'], how="left")
    return updated_user_review_df

In [65]:
sales_df = append_user_review_col_to_sales_df(user_reviews_df=user_reviews_df, sales_df=sales_df)

In [66]:
sales_df.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,user_review
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,8.1
1,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,8.4
2,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,8.2
3,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.38,9.23,6.50,2.90,30.01,8.5
4,Wii Play,Wii,2006.0,Misc,Nintendo,14.03,9.20,2.93,2.85,29.02,6.6


#### 8. DS preproccesing for all 3 data sets
this includes:
1. rounding user_review for classifiction problem
2. Transforming categroial columns with on hot encoding
3. 